# CREATE GEOMETRY and FLUXES FROM DATA

## Import

In [ ]:
# A 
import ast
# C
from collections import defaultdict
# G
import gc
import geopandas as gpd
# J
import json
# M
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool
# N
from numba import prange
import numpy as np
# O
import os
# P
import pandas as pd
# S
from shapely.geometry import box,LineString,Point,MultiPoint,MultiLineString,MultiPolygon,Polygon
from shapely.ops import unary_union
import socket
import sys
# T
from termcolor import  cprint
import time

# Project specific
# C
from ComputeGrid import *
# F
current_dir = os.path.join(os.getcwd()) 
mother_path = os.path.abspath(os.path.join(current_dir, os.pardir))
print('mother_path:', mother_path)
sys.path.append(os.path.join(mother_path, 'PreProcessing'))
sys.path.append(os.path.join(mother_path))
from FittingProcedures import *
# G
from GeometrySphere import *
from GenerateModifiedFluxesSimulation import *
from Grid import *
# H 
from Hexagon import *
if socket.gethostname()=='artemis.ist.berkeley.edu':
    sys.path.append(os.path.join('/home/alberto/LPSim','traffic_phase_transition','scripts','ServerCommunication'))
else:
    sys.path.append(os.path.join(os.getenv('TRAFFIC_DIR'),'scripts','ServerCommunication'))
from HostConnection import *
# M
from MainPolycentrism import *
from ModifyPotential import *
# O 
from ODfromfma import *
# P
from plot import *
from Polycentrism import *
from PolycentrismPlot import *
from PolygonSettings import *
from Potential import *
from PreprocessingObj import *



## BASIC PARAMS
gc.set_threshold(10000,50,50)
plt.rcParams.update({
    "text.usetex": False,
})



## Single City Variables

In [ ]:
# Always Needed
NameCity = "BOS"
# Computation Grid
ArgsComputationGrid = [(NameCity,TRAFFIC_DIR)]

## Multiple Cities Variables

In [ ]:
# Always Needed
NameCities = os.listdir(os.path.join(TRAFFIC_DIR,'data','carto'))
if len(NameCities)==0:
    cprint('No city found in the data/cart folder', 'red')
    sys.exit()
# Computation Grid
ArgsComputationGrid = [(NameCities[i],TRAFFIC_DIR) for i in range(len(list_cities))]

### Compute Grid

In [ ]:
for arg in ArgsComputationGrid:
    ComputeGrid(*arg)


# COMPUTE VECTOR FIELD

In [ ]:
name = 'BOS'
grid_size = 0.02
hexagon_resolution = 8
dir_grid = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))

## BASIC NEEDED OBJECTS
SFO_obj = GeometricalSettingsSpatialPartition(name,TRAFFIC_DIR)
bbox = SFO_obj.bounding_box
minx, miny, maxx, maxy = bbox
dir_geometries = os.path.join(TRAFFIC_DIR,'data','carto','{0}'.format(name))
grid = GetGrid(grid_size,SFO_obj.bounding_box,'EPSG:4326',dir_geometries)
grid = GetBoundariesInterior(grid,SFO_obj)
df_distance,FoundDistMat = GetDirectionMatrix(dir_geometries,grid_size)
Tij = GetODGrid(dir_geometries,str(grid_size))
lattice = nx.read_graphml(os.path.join(dir_grid,"centroid_lattice.graphml"))
#VectorField = pd.read_csv('/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/grid/0.02/VectorField.csv')
#Potential = pd.read_csv('/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/grid/0.02/PotentialDataFrame.csv')

# GET POTENTIAL AND VECTOR FIELD
VectorField = GetVectorField(Tij,df_distance)
VectorFieldDir = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))
lattice = GetPotentialLattice(lattice,VectorField)
lattice = SmoothPotential(lattice)
PotentialDataframe = ConvertLattice2PotentialDataframe(lattice)
PotentialDataframe = CompletePotentialDataFrame(VectorField,grid,PotentialDataframe)
SumPot = np.sum(PotentialDataframe['V_out']) 
NumGridEdge = grid[grid['relation_to_line']=='edge'].shape[0]




In [ ]:
ComputeInitPotential = True
ComputeInitVF = True
if ComputeInitVF:
    VectorField = GetVectorField(Tij,df_distance)
    VectorFieldDir = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))
    SaveVectorField(VectorField,VectorFieldDir)

if ComputeInitPotential:
    lattice = GetPotentialLattice(lattice,VectorField)
    lattice = SmoothPotential(lattice)
    PotentialDataframe = ConvertLattice2PotentialDataframe(lattice)
    PotentialDataframe = CompletePotentialDataFrame(VectorField,grid,PotentialDataframe)
    SavePotentialDataframe(PotentialDataframe,dir_grid)


# FIT GRAVITATIONAL MODEL

# MODIFY POTENTIAL

In [ ]:
import os
TRAFFIC_DIR = '/home/alberto/LPSim/traffic_phase_transition'
with open(os.path.join(TRAFFIC_DIR,'data','carto',name,'grid','0.02','origindest2grid.json'),'r') as f:
    OD2grid = json.load(f)

with open(os.path.join(TRAFFIC_DIR,'data','carto',name,'grid','0.02','grid2origindest.json'),'r') as f:
    grid2OD = json.load(f)
with open(os.path.join(TRAFFIC_DIR,'data','carto',name,'osmid2idx.json'),'r') as f:
    osmid2index = json.load(f)

list_peaks = [2,3,4,5,6,8,10,100,200,500] # [2,3,4,5,6,8,10,
InfoConfigurationPolicentricity = {num_peaks: defaultdict() for num_peaks in list_peaks}
Covarinaces = [1,2,4,8,10,15,20,30,50,100]
for cv in Covarinaces:
    for distribution in ['exponential']:#['gaussian','exponential']:#,'exponential']: # NOTE: There is some problem with the exponential distribution
        InfoCenters = {'center_settings': {"type":distribution},'covariance_settings':{"covariances":{"cvx":cv,"cvy":cv},"Isotropic": True,"Random": False}}
        #parameter_values = ((InfoConfigurationPolicentricity,grid,SFO_obj,Tij,df_distance,num_peaks) for num_peaks in list_peaks)
        for num_peaks in list_peaks:
            InfoConfigurationPolicentricity,UCI = ModifyMorphologyCity(InfoConfigurationPolicentricity,grid,SFO_obj,Tij,df_distance,lattice,num_peaks,TRAFFIC_DIR,name,grid_size,InfoCenters,fraction_fluxes = 200,verbose = True)
            if socket.gethostname()=='artemis.ist.berkeley.edu':
                SaveOd = "/home/alberto/LPSim/LivingCity/berkeley_2018/new_full_network"
            else:
                SaveOd = f'/home/aamad/Desktop/phd/traffic_phase_transition/data/carto/{name}/OD'
            OD_from_T_Modified(InfoConfigurationPolicentricity[num_peaks]['Tij'],
                               CityName2RminRmax,
                               name,
                               osmid2index,
                               grid2OD,
                               1,
                               SaveOd,
                               7,
                               8,
                               round(UCI,3))
#with multiprocessing.Pool(processes=multiprocessing.cpu_count() - 4) as pool:
    # Map the function to the parameter values
#    pool.map(ModifyMorphologyCity, parameter_values)
